In [ ]:
%matplotlib inline
import flopy.modflow as mfl
import flopy.utils as fut
import numpy as np
import matplotlib.pyplot as plt
import config

# SWI2 FloPy Island Function



### Interface flow below a long island
Consider steady interface flow below a long island. A cross-section across the island is shown in the figure. The width of the island is equal to $2W$. Flow is unconfined. Heads and elevations are measured with respect to mean sea level. The hydraulic conductivity of the aquifer is $k$. Groundwater recharge is equal to $N$.  

![](./img/island_interface.png)

Input variables:
* `k`: hydraulic conductivity [m/d]
* `D`: depth of impermeable bottom of aquifer [m]
* `c`: resistance of leaky sea bottom [d] (zero if no resistance)
* `rhof`: density of fresh water [kg/m$^3$]
* `rhos`: density of salt water [kg/m$^3$]
* `W`: half the width of the island [m]
* `N`: recharge rate [m/d]

In [ ]:
def island(nland, botm = -20, nsea=50,N=0.002, Q=0,zetainit=None,perlen=5000, nstp=500, tipslope=0.05,cland=40, delr = 5.,csea = 10
):
    ml = mfl.Modflow('gdansk', version='mf2005', exe_name=config.mfexe, model_ws='./data')
    # model dimensions
    nlay = 1
    nrow = 1
    ncol = nsea + nland + nsea
    delc = 1.
    nper, perlen, nstp = 1, perlen, nstp
    discret = mfl.ModflowDis(ml, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc,
                             top=0, botm=botm, steady=True, nper=nper, perlen=perlen, nstp=nstp)
    x = np.arange(0, ncol * delr, delr) - (ncol - 1) / 2 * delr
    # Basic Package
    ibound = np.ones((nlay, nrow, ncol))
    bas = mfl.ModflowBas(ml, ibound=ibound, strt=0.0)
    # LPF Package
    lpf = mfl.ModflowLpf(ml, hk=20., laytyp=0, layavg=0, layvka=1)
    # WELL Package
    if zetainit is not None:
        wel = mfl.ModflowWel(ml, stress_period_data = {0:[[0, 0, int((ncol - 1) / 2), -Q]]} )
    # Recharge on land
    rech = np.zeros((nrow, ncol))
    rech[0, nsea:nsea + nland] = N
    rch = mfl.ModflowRch(ml, rech=rech)
 

    # General head Boundary in the Sea
    lrchc = np.zeros((2 * nsea, 5))
    lrchc[:, 0] = 0
    lrchc[:, 1] = 0
    lrchc[:, 2] = np.hstack((np.arange(nsea), np.arange(nsea + nland, 2 * nsea + nland)))
    lrchc[:, 3] = 0.
    lrchc[:, 4] = delr * delc / csea
    ghb_data = {0:lrchc}
    ghb = mfl.ModflowGhb(ml, stress_period_data=ghb_data)

    # Output Control and Solver
    oc = mfl.ModflowOc(ml, save_every=1)
    pcg = mfl.ModflowPcg(ml)
    
    # SWI2
    if zetainit is None:
        z = np.zeros((nrow, ncol))
        z[0, nsea-14:nsea+14] = np.linspace(0, -28, 28)
        z[0, -nsea-14:-nsea+14] = np.linspace(-28, 0, 28)
        z[0, nsea+14: -nsea-14] = -28
        z = nlay * [z]
    else:
        z = [zetainit]
    isource = -2*np.ones((nrow, ncol), np.int)
    isource[0, nsea:nsea + nland] = 1
   
    
    #isource[nsea:nland+nsea]=1
    #
    swi = mfl.ModflowSwi2(ml, nsrf=1, istrat=1, 
                          toeslope=0.05, tipslope=tipslope, nu=[0, 0.025],
                          zeta=z, ssz=0.2, isource=isource, 
                          nsolver=1, iswizt=55)
    # Write and Run
    ml.write_input()
    ml.run_model(silent=True)
    # read model heads
    hfile = fut.HeadFile('./data/gdansk.hds')
    head = hfile.get_alldata()
    # read model zeta array[nt, nlay, nrow]
    zfile = fut.CellBudgetFile('./data/gdansk.zta')
    kstpkper = zfile.get_kstpkper()
    zeta = []
    for kk in kstpkper:
        zeta.append(zfile.get_data(kstpkper=kk, text='ZETASRF  1')[0])
    zeta = np.array(zeta)
    return x, zeta, head

## demo: using the island function
1) Specify the island dimensions

2) Call the function specifying recharge timesteps and optional swi parameters

3) plot the result

In [ ]:
# model dimensions cells
nland = 301
nsea = 50

# calculate SWI2 solution
x, zetainit, hinit = island(nland=nland, N=0.002, nsea=nsea, perlen=1000, nstp=20, tipslope=0.02)

plt.figure(figsize=(16, 4))
for i in range(10):
    plt.plot(x, zetainit[i, 0, 0], 'gray')
# plot starting interface
plt.plot(x, zetainit[0, 0, 0], 'b')
# plot final interface
plt.plot(x, zetainit[-1, 0, 0], 'r')    
plt.figure(figsize=(16, 4))
plt.subplot(121)
m = int((len(hinit[0, 0 ,0]) - 1) / 2)
plt.plot(hinit[:, 0, 0, m])
plt.title('head at center')
plt.subplot(122)
plt.title('zeta at the coast')
plt.plot(zetainit[:, 0, 0, nsea])



## demo: using the island function
how to specify the starting interface

In [ ]:
# model dimensions cells
nland = 301
nsea = 50
#copy last zeta from previous simulation
z0 = zetainit[-1].copy()
# calculate SWI2 solution
x, zetacheck, hcheck = island(zetainit=z0,nland=nland, N=0.002, nsea=nsea, perlen=10000, nstp=200, tipslope=0.02)

plt.figure(figsize=(16, 4))
for i in range(100):
    plt.plot(x, zetacheck[i, 0, 0], 'gray')
# plot starting interface
plt.plot(x, zetacheck[0, 0, 0], 'b')
# plot final interface
plt.plot(x, zetacheck[-1, 0, 0], 'r')    


plt.figure(figsize=(16, 4))
plt.subplot(121)
m = int((len(hcheck[0, 0 ,0]) - 1) / 2)
plt.plot(hcheck[:, 0, 0, m])
plt.title('head at center')
plt.subplot(122)
plt.title('zeta at the coast')
plt.plot(zetacheck[:, 0, 0, nsea])



## demo: comparing with the analytical solution
Call yesterdays islandinterface function
plot the result 
add the final zeta to the plot
adjust the xlimits

In [ ]:
# lets do this together....


### exercise 1: using the island function


1a) calculate the width of the island in the demo model 

1b) Approx. how many days does it take before the head and interface are in equilibrium?  


1c) Compare the SWI equilibrium with the analytical solution for the same case with a recharge of 0.001 m/d   


1d) now simulate and show the interface evolving from the 0.002 equilibrium to the 0.001 equilibrium show the result in a cross section,  